## Deployer de lil API 

Creación de workspace

Procura cambiar el nombre del grupo de recursos cada que lo corras para inicializar en otro y que sea más fácil 

Si vas a llamar un grupo de recursos ya existente, cambias create por get 
Puedes comentar la locación si solo vas a llamar a uno ya existente



In [6]:
import sklearn 
sklearn.__version__

'1.6.1'

In [ ]:
import json
from azureml.core.model import Model
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication

# Cargando el archivo con el ID
id = open('id_conf.json', 'r')
mi = json.load(id)

# El tenant ID está en AZURE BUSCANDO MICROSOFT ENTRA ID 
tenant_id = mi["tenant_id"] 

# Autenticación interactiva
interactive_auth = InteractiveLoginAuthentication(tenant_id=tenant_id)

# Autenticación con el ID del archivo
my_id = mi["id"]

# Creando el workspace llamando el ID
ws = Workspace.create(name="service",
                      subscription_id = my_id,
                      resource_group = "tca_grupo_recursos",
                      location = "centralindia"
                      )

# Registrando el modelo
model_name = "model_predictionnnnnn"
registered_model = Model.register(model_path="svm_model.pkl",
                                  model_name=model_name,
                                  workspace=ws)

Cleaning up past default Resource Group Deployments on the subscription to avoid limit of 10
Deleting past Resource Group Deployment with name: DeployResourceGroup-0841292edf
Deploying KeyVault with name servicekeyvaultd4aceadb9.
Deploying StorageAccount with name servicestorageb55585093c.
Deploying AppInsights with name serviceinsightsfdf5c3309.
Deployed AppInsights with name serviceinsightsfdf5c3309. Took 30.5 seconds.
Deploying Workspace with name service.
Deployed Workspace with name service. Took 40.4 seconds.
Registering model model_predictionnnnnn


El workspace que creas necesitará tener permisos de AZURE CONTAINER INSTANCES. 
Para esto: 
1) Entras al Grupo de Recursos en Azure recién creado y vas a la izquierda en Access control (IAM). 
2) Abajo encuentras la caja que dice 'Grant access to this resource' y presionas _Add role assignment_. 
3) Role: busca _Azure Container Instances_. Next; Members: _+ Select members_ y pegas el siguiente código y picas _Select_.

In [58]:
ws.get_details()['identity']['principal_id']

'4bd5d525-5724-427c-b2ce-d56b235c9efe'

pica _review + assign_ y vuelve a llamar al Workspace actualizado 

In [ ]:
# Creando el workspace llamando el ID
ws = Workspace.get(name="service",
                      subscription_id = my_id,
                      resource_group = "tca_grupo_recursos"
                      )

A continuación el código _score.py_. Ten en cuenta que para implementaciones en la nube ya no se puede llamar el dataset de manera local, sino que nesecitas que Azure lo haga. Según chatGPT el código es así: 

- model_path = os.path.join(os.getenv("AZUREML_MODEL_DIR"), "svm_model.pkl")
- model = joblib.load(model_path)

Nota: no olvides definir global model. 

Los print que están en el código son para debuggear, especialmente para la implementación local en Docker. 

In [ ]:
################ score.py ###################

scorepy = """
import os
import json
import pandas as pd
import joblib
from azureml.core.model import Model

def init():
  global model
  try:
    model_path = Model.get_model_path("svm_model1")
    model = joblib.load(model_path)
  except Exception as e:
    raise


def run(raw_data): 
  try: 
    # raw_data viene como JSON preprocesado desde la API
    if isinstance(raw_data, str):
      data_dict = json.loads(raw_data)
    else:
      data_dict = [raw_data]

    # Convertir directamente a DataFrame
    data = pd.DataFrame(data_dict)

    # Predecir directamente (el modelo ya tiene scaler dentro)
    result = model.predict(data) 
    
    if isinstance(raw_data, str):
      return json.dumps(result) 
    else:
      return json.dumps(int(result[0]))

  except Exception as e:
    print(str(e))
    return json.dumps(str(e))
"""

file_score = open("score.py", "w")
file_score.write(scorepy)
file_score.close()

Para la siguiente parte cambia el nombre del entorno virtual y del servicio cada vez que quieras hacer un intento diferente. 
En esta parte es importante recalcar que es donde se agregan las dependencias, por lo que también se podría agregar con un .txt o con un .yml. Por ejemplo: 
- v_env = Environment.from_conda_specification(name="comando_estelar_env1", file_path="environment.yml")

In [61]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

# Configurando el entorno de virtual
v_env = Environment("comando_estelar_env1")
v_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['pandas', 'scikit-learn==1.6.1', 'numpy'])

# Configurando el entorno de inferencia
inference_config = InferenceConfig(
                                environment=v_env,
                                entry_script="score.py",
                                )
aci_config = AciWebservice.deploy_configuration(cpu_cores=1.5, memory_gb=3) 

# Construyendo el servicio
service = Model.deploy(workspace=ws,
                       name='service1',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True, # Sobrescribir si ya existe
                       )

C:\Users\David\AppData\Local\Temp\ipykernel_26716\2981551881.py:18: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


Ya que está eso falta ver el deployment del servicio. Como se tarda tanto en correr está el código configurado para que enseñe todos los outputs y podamos ver qué tanto ha avanzado. 

In [ ]:
# Esperando a que el servicio se despliegue
service.wait_for_deployment(show_output=True)

# Guardando la URI del servicio
scoring_uri = service.scoring_uri
key = service.get_keys()[0]  # Obteniendo la clave de acceso

# Guardando la URI en un archivo JSON
scoreuri = json.dumps({"URI": [scoring_uri], "Key": key})
file = open("uri.json", "w")
file.write(scoreuri)
file.close()

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2025-06-06 00:25:51-06:00 Creating Container Registry if not exists..
2025-06-06 00:35:51-06:00 Registering the environment.
2025-06-06 00:35:53-06:00 Building image.

Adicional: Para correr esto en Docker de manera local y observar si el código funciona correctamente hacemos lo siguiente: 

In [ ]:
from azureml.core.webservice import LocalWebservice
local_config = LocalWebservice.deploy_configuration()
service = Model.deploy(ws, "local-svc", [registered_model], inference_config, local_config)
service.wait_for_deployment(show_output=True)
# Imprimiendo la URI del servicio
print("Local service URI:", service.scoring_uri)